# IO

In [ ]:
from PineBioML import IO

In [ ]:
# Path
input_folder = "./input/"
output_folder = "./output/"
export_title = "example_basic "

In [ ]:
x, y = IO.read_multiple_groups(
    file_path_list=[
        input_folder+ "example_group1.csv", # all samples from this file will have y = 0, because it is in the 1st place of the file path list.
        input_folder+ "example_group2.tsv", # all samples from this file will have y = 1, because it is in the 2nd place of the file path list.
    ],
    index_col=0
    )

In [ ]:
y.describe()

In [ ]:
target_label = 1.0

# Train test split

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.33, random_state=142, shuffle=True, stratify=y)

# Experiment

In [ ]:
from PineBioML.model.utils import Pine, sklearn_esitimator_wrapper, data_source

from PineBioML.preprocessing import Normalizer, Pass
from PineBioML.preprocessing.impute import knn_imputer, simple_imputer, iterative_imputer
from PineBioML.preprocessing.utils import feature_extension
from PineBioML.preprocessing.outlier import IsolationForest, simple_clip
from PineBioML.selection.classification import ensemble_selector
from PineBioML.model.supervised import Classification
from sklearn.linear_model import LogisticRegression


# define the experiment pipeline
experiment = [
    ("DataSource", {
        "first256": data_source(x_train.iloc[:, :50], x_valid.iloc[:, :50]),
        "others": data_source(x_train.iloc[:, 50:], x_valid.iloc[:, 50:])
    }),
    ("OutlierDetection", {
        "Simple clip": simple_clip(),
        "None": Pass()}),
    ("MissingValueProcessing", {
        "mean": simple_imputer(threshold=0.3333, strategy="mean")
    }),
    ("Standarization", {
        "StandardScaler": Normalizer(method="StandardScaler"),
    }),
    ("Selection",{
        "ensemble": ensemble_selector(z_importance_threshold=1),
        #"None": Pass()
    }),
    ("Modeling", {
        "LgisticRegression":
        sklearn_esitimator_wrapper(LogisticRegression(penalty=None)),
        "ElasticLogisticRegression":
        Classification.ElasticLogit_tuner(target="auc"),
        "rbf-SVM":
        Classification.SVM_tuner(target="acc"),
        "DecistionTree":
        Classification.DecisionTree_tuner(target="auc"),
        "RandomForest":
        Classification.RandomForest_tuner(target="auc"),
        "AdaBoost":
        Classification.AdaBoost_tuner(target="auc"),
        "XGBoost":
        Classification.XGBoost_tuner(target="auc"),
        "LightGBM":
        Classification.LightGBM_tuner(target="auc"),
    }),
]

In [ ]:
pine_automl = Pine(experiment, target_label = target_label, cv_result= True, evaluate_ncv = 3)
result = pine_automl.do_experiment(x_train, y_train, x_valid, y_valid)

In [ ]:
pine_automl.experiment_detail()

In [ ]:
# print the results, sort according to test_auc
result.sort_values("cv_auc")

In [ ]:
pine_automl.experiment_results(True).iloc[:, :12]

In [ ]:
# save the result
result.to_csv("./output/example_Pine_result.csv")

# Pick a model and save it

In [ ]:
# notice that the model is not fitted yet
model = pine_automl.recall_model(id = 2)

In [ ]:
# fit the model
model.fit(x_train, y_train)
y_valid_prob = model.predict_proba(x_valid)

### summary

In [ ]:
from PineBioML.report.utils import classification_summary

classification_summary(y_valid, y_valid_prob, target_label=target_label, save_fig=True, save_path="./output/images")

### save the model

IO.save_model(model, "./output/models/", "mode0", overide=True)